# Estimação do potencial de trabalho remoto no Brasil em 2019 

A estimação será feita usando, primeiramente, a base da PNAD Contínua suplementar de 2019 e, depois, a PNAD COVID

Foi adotado a hipótese de que a perda de potencial de trabalho remoto devido à falta de infraestrutura é a mesma na PNAD-COVID que na PNAD Contínua de 2019.

Trabalhar com as seguintes variáveis da PNAD Contínua: 
- V4010, que classifica as diversas ocupações, 
- a unidade da federação (UF), 
- gênero (V2007), 
- raça (V2010), 
- renda em todos os trabalhos (VD4019), 
- Setor Público (V4013, Código da principal atividade desse negócio/empresa),
- educação (VD3004), 
- formal/informal (combinação da variável VD4009 com a variável V4019) e 
- o peso amostral (V1032).

Da mesma forma, os equivalentes para a PNAD-COVID são:
- V1013, mês da pesquisa,
- V1012, semana da pesquisa,
- UF, unidade da federação,
- C007C, trabalho, cargo ou ocupação,
- A003, gênero,
- A004, raça,
- A005, escolaridade,
- C007D, setor público,
- C01012, renda, 
- C007, formal informal e
- o peso amostral, V1032.

Não esquecer de filtrar a base para os registros que a 'dummy_infra' igual à 1.

In [1]:
#pacotes 
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
#Importação dos dados da PNAD Contínua 2019 (suplementar)

#determina qual diretório usar para cada sistema operacional.
import os
if os.name == 'nt': #windows
    diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\suplementar\\'

else: #linux(ou mac)
    diretorio = '/home/matheus/Microdados PNAD/feather/'

filtro_pnadc = ['Trimestre','V4010', 'UF', 'V2007', 'V2010', 'VD4019', 'V4013', 'VD3004', 'VD4009', 'V4019', 'V1032']

pnadc_19 = pd.read_feather(fr'{diretorio}PNADC_2019_editado.feather', columns=filtro_pnadc)


#Importação dos códigos de ocupação traduzidos

ocupacoes = pd.read_excel('../Anexos/ocupacoes_traduzidas.xlsx')

In [3]:
#Importação dos dados da PNAD COVID

#determina qual diretório usar para cada sistema operacional.
import os
if os.name == 'nt': #windows
    diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\feather_covid\\'

else: #linux(ou mac)
    diretorio = '/home/matheus/Microdados PNAD/PNAD_COVID/'

#Leitura de todas as bases com as variáveis selecionadas e criação de variáveis dinamicamente.

filtro_pnad_covid = ['Ano', 'V1013', 'V1012', 'UF', 'C007C', 'A003', 'A004', 'A005', 'C007D', 'C01012', 'C007', 'V1032']

for i in range(5, 12):
    if i < 10:
        mes = '0' + str(i)
    else:
        mes = i
    print(f'Importando PNAD COVID {mes} 2020')
    globals()[f'pnad_covid_{mes}2020'] = pd.read_feather(fr'{diretorio}PNAD_COVID_{mes}2020.feather', columns=filtro_pnad_covid)

print('Fim da importação')

Importando PNAD COVID 05 2020
Importando PNAD COVID 06 2020
Importando PNAD COVID 07 2020
Importando PNAD COVID 08 2020
Importando PNAD COVID 09 2020
Importando PNAD COVID 10 2020
Importando PNAD COVID 11 2020
Fim da importação


In [4]:
#Concatena todas os dataframes da PNAD COVID

pnad_covid = pd.concat([
    pnad_covid_052020,
    pnad_covid_062020,
    pnad_covid_072020,
    pnad_covid_082020,
    pnad_covid_092020,
    pnad_covid_102020,
    pnad_covid_112020
    ])

### Junção dos códigos de ocupação com os dados da PNAD Contínua 2019

In [5]:
#Coluna com o código da ocupação Major das ocupações traduzidas
pnadc_19['COD_OCUP_MAJ'] = pnadc_19['V4010'].astype(str).str[:1]

In [6]:
ocupacoes['Cod'] = ocupacoes['Cod'].astype(str)

In [7]:
pnadc_19_remoto = pd.merge(pnadc_19, ocupacoes, left_on='COD_OCUP_MAJ', right_on='Cod', how='left')

In [8]:
pnadc_19_remoto.head(2)

,Trimestre,V4010,UF,V2007,V2010,VD4019,V4013,VD3004,VD4009,V4019,V1032,COD_OCUP_MAJ,code_ONET,Title_ONET,teleworkable,Cod,Titulação
0,1,3221,Rondônia,Mulher,Parda,1583.0,86001,Superior completo,Empregado no setor privado com carteira de tra...,NaN,149.43857,3,11-9141,"Property, Real Estate, and Community Associati...",1.0,3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO
1,1,3221,Rondônia,Mulher,Parda,1583.0,86001,Superior completo,Empregado no setor privado com carteira de tra...,NaN,149.43857,3,13-1021,"Buyers and Purchasing Agents, Farm Products",1.0,3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO


Até aqui, a PNAD Contínua com a dummy de infraestrutura e a base de ocupações traduzidas da O*NET para COD foram e unidas no dataframe <code>pnadc_19_remoto</code>.

### Classificação das ocupações na PNAD COVID e junção com a base de ocupações traduzidas da O*NET para COD

In [9]:
#Dicionário com ocupaçoes da pnad covid classificada pelo maior grupo da de ocupação da COD
dict_ocupacoes_pnad_covid = {
    'Técnico, profissional da saúde de nível médio':3,
    'Secretária, recepcionista':4,
    'Vendedor ambulante (feirante, camelô, comerciante de rua, quiosque)':9,
    'Professor da educação infantil, de ensino fundamental, médio ou superior,':2,
    'Pedreiro, servente de pedreiro, pintor, eletricista, marceneiro':7,
    'Outros':9, 
    'Porteiro, zelador':5, 
    'Padeiro, açougueiro e doceiro':7,
    'Motorista de caminhão (caminhoneiro),':8,
    'Artesão, costureiro e sapateiro':7,
    'Outra profissão de nível superior (advogado, engenheiro, contador, jornalista etc.)':2,
    'Segurança, vigilante, outro trabalhador dos serviços de proteção':5,
    'Faxineiro, auxiliar de limpeza etc. (em empresa pública ou privada),':9,
    'Diretor, gerente, cargo político ou comissionado':1,
    'Balconista, vendedor de loja':5,
    'Operador de máquinas, montador na indústria;':8,
    'Médico, enfermeiro, profissionais de saúde de nível superior':2,
    'Cabeleireiro, manicure e afins':5,
    'Auxiliar de escritório, escriturário':4,
    'Mecânico de veículos, máquinas industriais etc.':7,
    'Motorista (de aplicativo, de taxi, de van, de mototáxi, de ônibus)':8,
    'Cozinheiro e garçom (de restaurantes, empresas)':5,
    'Comerciante (dono do bar, da loja etc.)':5, 
    'Policial civil':5,
    'Empregado doméstico, diarista, cozinheiro (em domicílios particulares),':9,
    'Vendedor a domicílio, representante de vendas, vendedor de catálogo (Avon, Natura etc.)':5,
    'Outro técnico ou profissional de nível médio':3,
    'Agricultor, criador de animais, pescador, silvicultor e jardineiro':6,
    'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Eats, IFood, Rappy etc.)':9,
    'Auxiliar de produção, de carga e descarga;':9,
    'Pedagogo, professor de idiomas, música, arte e reforço escolar':2,
    'Motoboy':9,
    'Artista, religioso (padre, pastor etc.)':2,
    'Operador de Telemarketing':4,
    'Auxiliar da agropecuária (colhedor de frutas, boia fria, etc.)':9,
    'Cuidador de crianças, doentes ou idosos':5,
    'Entregador de mercadorias (de restaurante, de farmácia, de loja, Uber Esta, Iodo, Rapa etc.)':9
    }

In [19]:
ocupacoes[['Cod','Titulação']].sort_values('Cod').drop_duplicates()

,Cod,Titulação
841,0,OUTROS
85,1,DIRETORES E GERENTES
499,2,PROFISSIONAIS DAS CIÊNCIAS E INTELECTUAIS
662,3,TÉCNICOS E PROFISSIONAIS DE NÍVEL MÉDIO
852,4,TRABALHADORES DE APOIO ADMINISTRATIVO
1270,5,"TRABALHADORES DOS SERVIÇOS, VENDEDORES DOS COM..."
905,6,"TRABALHADORES QUALIFICADOS DA AGROPECUÁRIA, FL..."
1134,7,"TRABALHADORES QUALIFICADOS, OPERÁRIOS E ARTESÃ..."
1317,8,OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES
1335,9,OCUPAÇÕES ELEMENTARES


In [20]:
#Nova coluna com o código da ocupação Major das ocupações traduzidas
pnad_covid['COD_OCUP_MAJ'] = pnad_covid['C007C'].map(dict_ocupacoes_pnad_covid)
#converter para inteiro
pnad_covid['COD_OCUP_MAJ'].fillna(999, inplace=True)
pnad_covid['COD_OCUP_MAJ'] = pnad_covid['COD_OCUP_MAJ'].astype(int)


In [25]:
ocupacoes['Cod'] = ocupacoes['Cod'].astype(int)

In [26]:
#TODO: verificar pq não está funcionando
#pnad_covid_remoto = pd.merge(pnad_covid, ocupacoes, left_on='COD_OCUP_MAJ', right_on='Cod', how='left')

: 

: 

### Ajuste do trabalho remoto potencial 

O ajuste deve ser realizada através da seguinte equação:

$POT^K_{PNAD-COVID-INFRA} = POT^K_{PNAD-COVID} \times \frac{POT^K_{PNAD-INFRA}}{POT^K_{PNAD}}$

Em que:

- $POT^K_{PNAD-COVID-INFRA}$ é o trabalho remoto potencial da PNAD-COVID ajustado pela infraestrutura,

- $POT^K_{PNAD-COVID}$ é o potencial de trabalho remoto da PNAD-COVID sem o ajuste da infraestrutura,

- $POT^K_{PNAD-INFRA}$ é o potencial de trabalho remoto da PNAD Contínua de 2019 com ajuste de infraestrutura e 

- $POT^K_{PNAD}$ é o potencial de trabalho remoto calculado com base na PNAD Contínua de 2019 sem o ajuste de infraestrutura.


In [ ]:
pnadc_19_remoto.groupby('Titulação')[['V4010']].count()

,V4010
Titulação,
DIRETORES E GERENTES,871542
OCUPAÇÕES ELEMENTARES,1950088
OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES,1557504
OUTROS,279384
PROFISSIONAIS DAS CIÊNCIAS E INTELECTUAIS,5116839
TRABALHADORES DE APOIO ADMINISTRATIVO,502793
"TRABALHADORES DOS SERVIÇOS, VENDEDORES DOS COMÉRCIOS E MERCADOS",2828732
"TRABALHADORES QUALIFICADOS DA AGROPECUÁRIA, FLORESTAIS, DA CAÇA E DA PESCA",1651179
"TRABALHADORES QUALIFICADOS, OPERÁRIOS E ARTESÃOS DA CONSTRUÇÃO, DAS ARTES MECÂNICAS E OUTROS OFÍCIOS",3897330


In [ ]:
pnadc_19_remoto[pnadc_19_remoto['dummy_infra'] == 1].groupby('Titulação')['V4010'].count() / pnadc_19_remoto.groupby('Titulação')['V4010'].count()

Titulação
DIRETORES E GERENTES                                                                                    0.769987
OCUPAÇÕES ELEMENTARES                                                                                   0.205841
OPERADORES DE INSTALAÇÕES E MÁQUINAS E MONTADORES                                                       0.386552
OUTROS                                                                                                  0.755262
PROFISSIONAIS DAS CIÊNCIAS E INTELECTUAIS                                                               0.841648
TRABALHADORES DE APOIO ADMINISTRATIVO                                                                   0.643094
TRABALHADORES DOS SERVIÇOS, VENDEDORES DOS COMÉRCIOS E MERCADOS                                         0.429938
TRABALHADORES QUALIFICADOS DA AGROPECUÁRIA, FLORESTAIS, DA CAÇA E DA PESCA                              0.175733
TRABALHADORES QUALIFICADOS, OPERÁRIOS E ARTESÃOS DA CONSTRUÇÃO, DAS ARTES MECÂNICAS E 